# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## 1.Завантаження залежних та незалежних вибірок з текстових файлів. 

In [18]:
path = "../data_lab4/dep/4Prak.txt"
columns = ['x','y']
df = pd.read_csv(path, delimiter=" ", header=None, names=columns)
df.head()

,x,y
0,86,187
1,78,187
2,642,653
3,192,336
4,571,466


## Критерії перевірки однорідності двох залежних вибірок: критерії перевірки рівності дисперсій та середніх + критерій знакових рангів Вілкоксона

In [27]:
class DependentSamples:
    @classmethod
    def create_thrird_sample(cls, df):
        df['z'] = df['x']-df['y']

        z_mean = np.mean(df['z'])
        squared_z_std = np.std(df['z'], ddof=1)
       
        return DependentSamples.calculate_t(len(df)-1, z_mean, squared_z_std)

    @classmethod
    def calculate_t(cls, length, z_mean, squared_z_std):
        return z_mean*np.sqrt(length)/squared_z_std
        
    @staticmethod
    def check_for_homogeneity(df):
        return DependentSamples.create_thrird_sample(df)

In [28]:
DependentSamples.check_for_homogeneity(df)

0.44087361536851716

## Критерії перевірки однорідності двох незалежних вибірок: критерії перевірки рівності дисперсій та середніх + ранговий критерій, заданий індивідуальним варіантом.

### Check wheather distribution normal or not

In [13]:
alpha = 0.05
# Shapiro-Wilk test for 'x'
statistic_x, p_value_x = stats.shapiro(df['x'])
is_normal_x = p_value_x > alpha

# Shapiro-Wilk test for 'y'
statistic_y, p_value_y = stats.shapiro(df['y'])
is_normal_y = p_value_y > alpha

if is_normal_x and is_normal_y:
    is_normal =True;
else:
    is_normal =False;

print(is_normal)

True


### Checking the equality of variances

In [ ]:
x_mean = np.mean(df['x'])
y_mean = np.mean(df['y'])
squared_x_std = np.std(df['x'], ddof=1)**2
squared_y_std = np.std(df['y'], ddof=1)**2
print("Mean for 1 column: "+ str(x_mean))
print("Mean for 2 column: "+ str(y_mean))
print("STD for 1 column: "+ str(squared_x_std))
print("STD for 2 column: "+ str(squared_y_std))

In [ ]:
# Коефіцієнти для формули
C0 = 2.515_517
C1 = 0.802_853
C2 = 0.010_328
D1 = 1.432_788
D2 = 0.189_265_9
D3 = 0.001_308

class Quantile:
    
    @classmethod
    def calculate_quantile(cls, p):
        if p > 0.5:
            t = np.sqrt(-2 * np.log(1 - p))
            quantile = (t - ((C0 + C1 * t + C2 * t ** 2) / (1 + D1 * t + D2 * t ** 2 + D3 * t ** 3)))
        else:
            t = np.sqrt(-2 * np.log(p))
            quantile = -(t - ((C0 + C1 * t + C2 * t ** 2) / (1 + D1 * t + D2 * t ** 2 + D3 * t ** 3)))

        return quantile
    @staticmethod
    def student_quantile(self, p, v):
        up = Quantile.calculate_quantile(p)
        tpv = up + (1 / v) * (1 / 4) * (up ** 3 + up) + (1 / v ** 2) * (1 / 96) * (
            5 * up ** 5 + 16 * up ** 3 + 3 * up) + (1 / v ** 3) * (1 / 384) * (
                  3 * up ** 7 + 19 * up ** 5 + 17 * up ** 3 - 15 * up) + (1 / v ** 4) * (1 / 92_160) * (
                  79 * up ** 9 + 779 * up ** 7 + 1_482 * up ** 5 - 1_920 * up ** 3 - 945 * up)
        return tpv
    
    @staticmethod
    def fisher_quantile(p ,v1, v2):
        up = Quantile.calculate_quantile(p)   

        sigma = 1/v1 + 1/v2
        delta = 1/v1 - 1/v2
        
        z = (up * np.sqrt(sigma / 2) -
         (1 / 6) * delta * (up**2 + 2) +
         np.sqrt(sigma / 2) * (sigma / 24 * (up**2 + 3 * up) + (1 / 72) * (delta**2 / sigma) * (up**3 + 11 * up)) -
         (sigma * delta / 120) * (up**4 + 9 * up**2 + 8) +
         (delta**3 / (3240 * sigma)) * (3 * up**4 + 7 * up**2 - 16) +
         np.sqrt(sigma / 2) * (sigma**2 / 1920 * (up**5 + 20 * up**3 + 15 * up)) +
         (delta**4 / 2880) * (up**5 + 44 * up**3 + 183 * up) +
         (delta**5 / (155520 * sigma**2)) * (9 * up**5 + 284 * up**3 + 1513 * up)
        )

        return np.exp(2*z)

#### Calculate P value with CDF(using student quantile)

### Calculate paired t-test

In [ ]:
p = 2(1-Quantile.student_quantile()
v1 = df['x'].count() - 1
v2 = df['y'].count() - 1 

Quantile.fisher_quantile(0.95,v1,v2)


In [ ]:
F = (squared_x_std/squared_y_std)
if(is_normal)
    2
elif(f>1)
print("F equalse: " + str(F))
    2